In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from textblob import TextBlob
from transformers import pipeline

In [ ]:

def get_sentiment_with_textblob(sentence):
    analysis = TextBlob(sentence)
    return analysis.sentiment.polarity

In [ ]:
def get_sentiment_with_roberta(sentence):
    sentiment_pipeline = pipeline('sentiment-analysis', model='roberta-large-mnli')
    result = sentiment_pipeline(sentence)
    return result[0]["score"]

In [ ]:
def get_textblob_sentiment(sentence):
    return get_sentiment_with_textblob(sentence)

def get_roberta_sentiment(sentence):
    return get_sentiment_with_roberta(sentence)

In [ ]:
def process_character_sentiments(character_name, sentences):
    sentences_textblob = []
    sentences_roberta = []
    
    for sentence in sentences:
        sentiment_with_textblob = get_sentiment_with_textblob(sentence)  
        sentences_textblob.append(sentiment_with_textblob)

        sentiment_with_roberta = get_sentiment_with_roberta(sentence)
        sentences_roberta.append(sentiment_with_roberta)
    
    mean_sentiment_textblob = np.mean(sentences_textblob) if sentences_textblob else None
    mean_sentiment_roberta = np.mean(sentences_roberta) if sentences_roberta else None
    
    return mean_sentiment_textblob, mean_sentiment_roberta

data_textblob = []  
data_roberta = []
list_charatcter = df_cleaned["Character_Name"]

for index, character_name in enumerate(list_characters):
    movie_id = df_cleaned["Movie_ID"].iloc[index]
    list_sentences_by_character = df_cleaned["Sentences"].iloc[index]
    
    mean_sentiment_textblob, mean_sentiment_roberta = process_character_sentiments(character_name, list_sentences_by_character)

    data_textblob.append({
        "Movie_ID": movie_id,
        "Character_Name": character_name,
        "Sentiment": mean_sentiment_textblob
    })

    data_roberta.append({
        "Movie_ID": movie_id,
        "Character_Name": character_name,
        "Sentiment": mean_sentiment_roberta
    })

df_sentiment_analysis_textblob = pd.DataFrame(data_textblob)
df_sentiment_analysis_roberta = pd.DataFrame(data_roberta)

In [ ]:
df_sentiment_analysis = pd.merge(df_sentiment_analysis_textblob, df_sentiment_analysis_roberta, 
                     on=["Character_Name", "Movie_ID"], 
                     suffixes=('_textblob', '_roberta'))